In [ ]:
def equalize(vct_out_sync, vct_h, pow_noise, vct_ak):
  # LMS equalizer and symbol detector

  [vct_c_opt, vct_alpha, matrix_phi] = fun_c_opt(N//2+1, downsample(vct_h, SAMPLESxSYM), pow_noise) # Theoric coefficients
  
  selected_delay = 1   # ver de implementar selected_delay = best_delay(vct_out_sync, 15, pow_noise, vct_h, True);
  vct_c = np.zeros(N, dtype=np.complex_) # Equalizer coefficients initialization
  matrix_c = np.zeros((N, SYMBOLS), dtype=np.complex_)
  vct_equalized = np.array([]); vct_detected_symbs = np.array([])
  variable_step = STEP; step_change = SYMBOLS//4

  vct_Ek = np.zeros((SYMBOLS, 1), dtype=np.complex_) # Symbols error vector for plotting
  Ek_SAMPLES = SYMBOLS//10 # Symbols error quantity for the moving average filter
  vct_MSE_symb = np.zeros((SYMBOLS-Ek_SAMPLES+1, 1)) # Symbols MSE vector for plotting

  vct_Q = np.zeros((SYMBOLS, 1), dtype=np.complex_) # Coefficients error vector for plotting
  Q_SAMPLES = SYMBOLS//10 # Coefficients error quantity for the moving average filter
  vct_MSE_coef = np.zeros((SYMBOLS-Q_SAMPLES+1, 1)) # Coefficients MSE vector for plotting

  for it_mc in np.arange(0, len(vct_out_sync)-N+1):
    
    Ak_delayed = vct_ak[selected_delay + N//2 + it_mc]
    vct_rk = vct_out_sync[0 +it_mc : N + it_mc]
    Yk_equalized = vct_c.T @ vct_rk

    # Detection of the symbol
    Ak_detected = detect(Yk_equalized, form = 'scalar')
    # print('equalized: ', Yk_equalized, 'detected: ', Ak_detected)

    # Symbols error calculation 
    if it_mc < TRAINING_END:
      Ek = Ak_delayed - Yk_equalized # training error (eq. 9.7)
    else:
      Ek = Ak_detected - Yk_equalized # real error

    ### Coefficients error calculation ###########################
    Q = (vct_c - vct_c_opt).sum() # (eq. 9.31)

    # Coefficients update 
    vct_c += variable_step * Ek * np.conj(vct_rk) # (eq. 9.53)

    # Update of the variable factor Beta: 4 steps 
    if it_mc > step_change:
      variable_step *= 0.5
      step_change += step_change

    # Construction of arrays for plotting 
    matrix_c[:, it_mc] = vct_c.T
    vct_equalized = np.append(vct_equalized, Yk_equalized) # ver, tiene N menos simbolos
    vct_detected_symbs = np.append(vct_detected_symbs, Ak_detected)
    vct_Ek[it_mc] = Ek
    vct_Q[it_mc] = Q

    # Moving Average Filters 
    # For MSE among transmitted and detected symbols
    if it_mc >= Ek_SAMPLES:
      vct_MSE_symb[it_mc-Ek_SAMPLES] = ( (vct_Ek[it_mc-Ek_SAMPLES : it_mc+1] \
      * np.conj( vct_Ek[it_mc-Ek_SAMPLES : it_mc+1] )).sum(axis=0)/Ek_SAMPLES ).real

    # For MSE among actual and optimal coefficients
    if (it_mc >= Q_SAMPLES):
      vct_MSE_coef[it_mc-Q_SAMPLES] = ( (vct_Q[it_mc-Q_SAMPLES : it_mc+1] \
      * np.conj( vct_Q[it_mc-Q_SAMPLES : it_mc+1] )).sum(axis=0)/Q_SAMPLES ).real

  return [vct_detected_symbs, vct_c, Ek_SAMPLES, Q_SAMPLES, vct_MSE_symb, vct_MSE_coef, matrix_c, vct_c_opt]